In [66]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re
pd.set_option('display.max_columns', None)

In [67]:
# location of raw data files
input_path = "../datasets/raw/"

In [68]:
#read PROCEDUREEVENTS_MV.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'PROCEDUREEVENTS_MV.csv',dtype = "object")

In [69]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258066 entries, 0 to 258065
Data columns (total 25 columns):
ROW_ID                        258066 non-null object
SUBJECT_ID                    258066 non-null object
HADM_ID                       258066 non-null object
ICUSTAY_ID                    257978 non-null object
STARTTIME                     258066 non-null object
ENDTIME                       258066 non-null object
ITEMID                        258066 non-null object
VALUE                         258066 non-null object
VALUEUOM                      258066 non-null object
LOCATION                      52612 non-null object
LOCATIONCATEGORY              52612 non-null object
STORETIME                     258066 non-null object
CGID                          258066 non-null object
ORDERID                       258066 non-null object
LINKORDERID                   258066 non-null object
ORDERCATEGORYNAME             258066 non-null object
SECONDARYORDERCATEGORYNAME    0 non-null ob

In [70]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,LOCATIONCATEGORY,STORETIME,CGID,ORDERID,LINKORDERID,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE
0,379,29070,115071,232563,2145-03-12 23:04:00,2145-03-12 23:05:00,225401,1,None,NaN,NaN,2145-03-12 23:04:00,20855,5092605,5092605,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
1,380,29070,115071,232563,2145-03-12 23:04:00,2145-03-12 23:05:00,225454,1,None,NaN,NaN,2145-03-12 23:04:00,20855,2538599,2538599,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
2,381,29070,115071,232563,2145-03-12 23:05:00,2145-03-18 20:01:00,225792,8456,hour,NaN,NaN,2145-03-18 20:01:39,20889,2383835,2383835,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
3,382,29070,115071,232563,2145-03-12 23:36:00,2145-03-12 23:37:00,225402,1,None,NaN,NaN,2145-03-12 23:36:00,20855,9355062,9355062,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
4,383,29070,115071,232563,2145-03-13 01:27:00,2145-03-16 16:00:00,224560,5193,min,Right IJ,Invasive Venous,2145-03-16 17:57:00,20855,466169,466169,Invasive Lines,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN


In [71]:
df = df.drop(labels = ['STARTTIME','ENDTIME','CGID','ORDERID','LINKORDERID',
                  'SECONDARYORDERCATEGORYNAME','ISOPENBAG','CONTINUEINNEXTDEPT',
                  'CANCELREASON','COMMENTS_EDITEDBY','COMMENTS_CANCELEDBY','COMMENTS_DATE'], axis = 'columns')

In [72]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,VALUE,VALUEUOM,LOCATION,LOCATIONCATEGORY,STORETIME,ORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,STATUSDESCRIPTION
0,379,29070,115071,232563,225401,1,None,NaN,NaN,2145-03-12 23:04:00,Procedures,Electrolytes,FinishedRunning
1,380,29070,115071,232563,225454,1,None,NaN,NaN,2145-03-12 23:04:00,Procedures,Electrolytes,FinishedRunning
2,381,29070,115071,232563,225792,8456,hour,NaN,NaN,2145-03-18 20:01:39,Ventilation,Task,FinishedRunning
3,382,29070,115071,232563,225402,1,None,NaN,NaN,2145-03-12 23:36:00,Procedures,Electrolytes,FinishedRunning
4,383,29070,115071,232563,224560,5193,min,Right IJ,Invasive Venous,2145-03-16 17:57:00,Invasive Lines,Task,FinishedRunning


In [73]:
#read PROCEDUREEVENTS_MV.csv file
df1 = pd.read_csv(filepath_or_buffer=input_path+'D_ITEMS.csv',dtype = "object")

In [74]:
df1.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12487 entries, 0 to 12486
Data columns (total 10 columns):
ROW_ID          12487 non-null object
ITEMID          12487 non-null object
LABEL           12483 non-null object
ABBREVIATION    2992 non-null object
DBSOURCE        12487 non-null object
LINKSTO         12487 non-null object
CATEGORY        6049 non-null object
UNITNAME        1234 non-null object
PARAM_TYPE      2992 non-null object
CONCEPTID       0 non-null object
dtypes: object(10)
memory usage: 975.6+ KB


In [75]:
df1.head()

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [76]:
df = df.merge(df1[['ITEMID','LABEL']], how='left',left_on='ITEMID',right_on='ITEMID')

In [77]:
df = df.fillna('NaN')

In [78]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,VALUE,VALUEUOM,LOCATION,LOCATIONCATEGORY,STORETIME,ORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,STATUSDESCRIPTION,LABEL
0,379,29070,115071,232563,225401,1,None,NaN,NaN,2145-03-12 23:04:00,Procedures,Electrolytes,FinishedRunning,Blood Cultured
1,380,29070,115071,232563,225454,1,None,NaN,NaN,2145-03-12 23:04:00,Procedures,Electrolytes,FinishedRunning,Urine Culture
2,381,29070,115071,232563,225792,8456,hour,NaN,NaN,2145-03-18 20:01:39,Ventilation,Task,FinishedRunning,Invasive Ventilation
3,382,29070,115071,232563,225402,1,None,NaN,NaN,2145-03-12 23:36:00,Procedures,Electrolytes,FinishedRunning,EKG
4,383,29070,115071,232563,224560,5193,min,Right IJ,Invasive Venous,2145-03-16 17:57:00,Invasive Lines,Task,FinishedRunning,PA Catheter


In [79]:
# Merge ITEMID with respective VALUE and UOM
df['VALUE_UOM'] = df['ITEMID']+'-'+df['VALUE']+'-'+df['VALUEUOM']

# if VALUE is missing map VALEU_UOM as 'NaN'
df.loc[df['VALUE']=='NaN','VALUE_UOM'] = 'NaN' 

df = df.drop(labels = ['VALUE','VALUEUOM', 'ITEMID'], axis = 'columns' )

In [86]:
# Remove special charecters from the free text columns
df['LABEL'] = df['LABEL'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['LOCATION'] = df['LOCATION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['LOCATIONCATEGORY'] = df['LOCATIONCATEGORY'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['ORDERCATEGORYNAME'] = df['ORDERCATEGORYNAME'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['ORDERCATEGORYDESCRIPTION'] = df['ORDERCATEGORYDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['STATUSDESCRIPTION'] = df['STATUSDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

In [87]:
# create procedure events
df['procedureevents'] = df['LABEL']+' '+df['VALUE_UOM']+' '+df['ORDERCATEGORYNAME']+' '+df['ORDERCATEGORYDESCRIPTION']+' '+df['STATUSDESCRIPTION']+' '+df['LOCATION']+' '+df['LOCATIONCATEGORY']

In [88]:
# drop redundant columns
df = df.drop(labels=['LABEL','VALUE_UOM','LOCATION','LOCATIONCATEGORY',
                     'ORDERCATEGORYNAME','ORDERCATEGORYDESCRIPTION',
                     'STATUSDESCRIPTION'], axis = 'columns')

In [89]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STORETIME,procedureevents
0,379,29070,115071,232563,2145-03-12 23:04:00,Blood Cultured 225401-1-None Procedures Electr...
1,380,29070,115071,232563,2145-03-12 23:04:00,Urine Culture 225454-1-None Procedures Electro...
2,381,29070,115071,232563,2145-03-18 20:01:39,Invasive Ventilation 225792-8456-hour Ventilat...
3,382,29070,115071,232563,2145-03-12 23:36:00,EKG 225402-1-None Procedures Electrolytes Fini...
4,383,29070,115071,232563,2145-03-16 17:57:00,PA Catheter 224560-5193-min Invasive Lines Tas...


In [90]:
df['procedureevents'] = df['procedureevents'].map(lambda x: x.split())

In [91]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STORETIME,procedureevents
0,379,29070,115071,232563,2145-03-12 23:04:00,"[Blood, Cultured, 225401-1-None, Procedures, E..."
1,380,29070,115071,232563,2145-03-12 23:04:00,"[Urine, Culture, 225454-1-None, Procedures, El..."
2,381,29070,115071,232563,2145-03-18 20:01:39,"[Invasive, Ventilation, 225792-8456-hour, Vent..."
3,382,29070,115071,232563,2145-03-12 23:36:00,"[EKG, 225402-1-None, Procedures, Electrolytes,..."
4,383,29070,115071,232563,2145-03-16 17:57:00,"[PA, Catheter, 224560-5193-min, Invasive, Line..."


In [92]:
df = df.drop(labels=['ROW_ID'], axis = 'columns')

In [93]:
# create output path
mypath_output = "../datasets/procedureevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [94]:
# export the dataframe to JSON format
df.to_json(mypath_output+"procedureevents.json",orient = 'records') 